Math 5750/6880: Mathematics of Data Science \
Project 3

In [1]:
import numpy as np
from sklearn import datasets
from sklearn.datasets import load_iris, make_moons, load_breast_cancer, fetch_california_housing
from sklearn.datasets import get_data_home
# from sklearn.linear_model import Perceptron
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.neural_network import MLPClassifier, MLPRegressor
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.linear_model import LinearRegression
from scipy import io #will use this to load matlab data

import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from mpl_toolkits.mplot3d import Axes3D

# 1. Fashion-MNIST image classification using sklearn

In [4]:
from tensorflow.keras.datasets import fashion_mnist
from sklearn.preprocessing import StandardScaler

# Load Fashion-MNIST
# Classes (0-9): T-shirt/top, Trouser, Pullover, Dress, Coat, Sandal, Shirt, Sneaker, Bag, Ankle boot
(X_train1, y_train1), (X_test1, y_test1) = fashion_mnist.load_data()
X_train1 = X_train1.reshape(len(X_train1), -1)
X_test1  = X_test1.reshape(len(X_test1), -1)

# Scale features
scaler = StandardScaler()
X_train1 = scaler.fit_transform(X_train1)
X_test1 = scaler.transform(X_test1)

In [5]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# your code here

https://docs.pytorch.org/tutorials/beginner/basics/tensorqs_tutorial.html

# 3. Fashion-MNIST image classification  using pytorch

In [6]:
import numpy as np
from tensorflow.keras.datasets import fashion_mnist
import torch
from torch.utils.data import TensorDataset, DataLoader

# Load Fashion-MNIST
# Classes (0-9): T-shirt/top, Trouser, Pullover, Dress, Coat, Sandal, Shirt, Sneaker, Bag, Ankle boot
(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()

# scale to [0,1], add channel dimension -> (N, 1, 28, 28)# Adding a channel dimension changes image tensors from (N, 28, 28) to (N, 1, 28, 28), matching CNN input shape requirements.
X_train = (X_train.astype("float32") / 255.0)[:, None, :, :]
X_test  = (X_test.astype("float32")  / 255.0)[:,  None, :, :]

y_train = y_train.astype(np.int64)
y_test  = y_test.astype(np.int64)

# train/val split: last 10k of train as validation
X_tr, X_val = X_train[:50000], X_train[50000:]
y_tr, y_val = y_train[:50000], y_train[50000:]

# wrap in PyTorch TensorDatasets and DataLoaders
train_ds = TensorDataset(torch.from_numpy(X_tr),  torch.from_numpy(y_tr)) # Tensors can be created from NumPy arrays: torch.from_numpy(np_array)
val_ds   = TensorDataset(torch.from_numpy(X_val), torch.from_numpy(y_val)) # DataLoader is an iterable that abstracts this complexity for us in an easy API.
test_ds  = TensorDataset(torch.from_numpy(X_test), torch.from_numpy(y_test))# DataLoader 是一个可迭代对象，通过简单的 API 帮助我们自动处理数据加载的复杂过程，方便高效地读取批量数据

train_loader = DataLoader(train_ds, batch_size=128, shuffle=True)# A batch is like a small “sub-dataset” sampled from the full dataset that the model processes together in one training step.
val_loader   = DataLoader(val_ds,   batch_size=256, shuffle=False)
test_loader  = DataLoader(test_ds,  batch_size=256, shuffle=False)

In [7]:
import torch
import torch.nn as nn
import torch.optim as optim

# In colab, you should ``change runtime type'' to GPU.
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# your code here

Using device: cuda


In [8]:
import tensorflow as tf

# "MLP" for lbfgs (Quasi-Newton methods)

https://docs.pytorch.org/docs/stable/nn.html

In [9]:
import torch.nn.functional as F

In [ ]:
class MLPModel(nn.Module): # inherent from nn.Modle
  def __init__(self):
    super().__init__(): # # calls nn.Module constructor
    self.fc1 = nn.Linear(28*28, 256) #same indent as calling the parent constructor
                  # implicitly defines the input dimension


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# Define CNN model
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        # Input: (N, 1, 28, 28)
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)  # -> (N, 32, 28, 28)
        self.pool1 = nn.MaxPool2d(2, 2)  # -> (N, 32, 14, 14), redcue dim
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1) # -> (N, 64, 14, 14)
        self.pool2 = nn.MaxPool2d(2, 2)                          # -> (N, 64, 7, 7)
        self.fc1 = nn.Linear(64 * 7 * 7, 100)
        self.fc2 = nn.Linear(100, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool1(x)
        x = F.relu(self.conv2(x))
        x = self.pool2(x)
        x = x.view(x.size(0), -1)  # Flatten
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Create model instance
model_1 = CNNModel()
print(model_1)


CNNModel(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=3136, out_features=100, bias=True)
  (fc2): Linear(in_features=100, out_features=10, bias=True)
)


In [ ]:
import time

In [ ]:
# Define loss function and optimizer
loss_fn = nn.CrossEntropyLoss()  # same as SparseCategoricalCrossentropy(from_logits=True)
optimizer_1 = optim.Adam(model_1.parameters(), lr=1e-3)

In [ ]:

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_1 = model_1.to(device)  # ✅ move model weights to GPU

# Get one batch and move to same device
start_t_CNN_total_1 = time.time()
for epoch in range(5):
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        optimizer_1.zero_grad()
        preds = model_1(X_batch)
        loss = loss_fn(preds, y_batch)
        loss.backward()
        optimizer_1.step()
    print(f"Epoch {epoch+1}/5 completed")
end_t_CNN_total_1 = time.time()
total_time_trained=end_t_CNN_total_1-start_t_CNN_total_1
print('total time trained by Pytorch:',total_time_trained)


Epoch 1/5 completed
Epoch 2/5 completed
Epoch 3/5 completed
Epoch 4/5 completed
Epoch 5/5 completed
total time trained by Pytorch: 6.523069381713867


In [ ]:
from sklearn.metrics import f1_score, accuracy_score

In [ ]:
# ===== VALIDATION =====
model_1.eval()
all_preds, all_labels = [], []

with torch.no_grad():
    for X_batch, y_batch in val_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        preds = model_1(X_batch)
        predicted = preds.argmax(1)
        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(y_batch.cpu().numpy())

# Compute metrics
acc = accuracy_score(all_labels, all_preds)
f1 = f1_score(all_labels, all_preds, average='macro')

print(f"Validation Accuracy: {acc:.4f}")
print(f"Validation F1 Score (macro): {f1:.4f}")

Validation Accuracy: 0.9145
Validation F1 Score (macro): 0.9139
